In [1]:
import json
import numpy as np
import torch
from torch import nn
from torch.distributions import Normal
from torch.nn import functional as F
from torch.optim import Adam, SGD
import random
from datetime import datetime
import matlab.engine
import json
import requests


from numpy import random

In [3]:
import matlab.engine

eng = matlab.engine.start_matlab()
eng.cd("../matlab_env/")

'/Users/valeriytashchilin/Dropbox (Личный)/Университет/DataSpell/rl_pss/notebooks'

In [11]:
%%time
out = eng.simWrapper('scenariosNormState', 'IntMaxDeltaWs', 0, 4)
print(out)

CPU times: user 24.4 ms, sys: 24.9 ms, total: 49.3 ms
Wall time: 2min 14s


In [54]:
with open("../trajectory_log.txt", 'r') as f:
    data = json.load(f)

In [57]:
[x for x in data[0]['data'][0]]

['t', 'action', 'pureAction', 'probability', 'reward', 'state']

In [52]:
 torch.tensor(data['data'][0]['state'], dtype=torch.float).shape

torch.Size([4, 60])

In [13]:
def create_trajectory(log_path: str):
    with open(log_path, 'r') as f:
        data = json.load(f)
    trajectories = []
    for exp in data:
        traj = []
        for ep in exp['data']:
            state = [x[-1] for x in ep['state']]
            traj.append((state, ep['pureAction'], ep['reward'], ep['probability'][0], ep['probability'][1]))
        trajectories.append(traj)
    return trajectories

In [14]:
parse = create_trajectory("../trajectory_log.txt")

In [28]:
sample = torch.Tensor(random.random((1, 4, 60)))
print(sample.shape)

torch.Size([1, 4, 60])


In [122]:
mean = torch.nn.Sequential(
    torch.nn.Linear(60,60),
    torch.nn.ReLU(),
    torch.nn.Linear(60, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 5),
    torch.nn.ReLU(),
)

In [123]:
print(mdl3(sample).shape)
print(mdl11(sample).shape)
mdlj = torch.cat([mdl3(sample), mdl11(sample)], dim=1)
print(mdl5(mdlj).shape)
print(mean(mdl5(mdlj)).shape)

torch.Size([1, 8, 60])
torch.Size([1, 8, 60])
torch.Size([1, 1, 60])
torch.Size([1, 1, 5])


In [ ]:
mdl5 = torch.nn.Sequential(
    torch.nn.Conv1d(16, 8, kernel_size=5, padding=2),
    torch.nn.ReLU(),
    torch.nn.Conv1d(8, 4, kernel_size=5, padding=2),
    torch.nn.ReLU(),
)

In [10]:
request = eng.simWrapper('scenarios_NormalStates', 'IntMaxDeltaWs', 1.0e+06, 1, "./log/")

In [37]:
agent = PPO(torch.tensor([40, 0.1, 0.1, 1, 1]), 5)

In [35]:
state_t = torch.tensor(state, dtype=torch.float)

In [54]:
agent.act(state_t)

(tensor([[10.9163,  0.0632,  0.0250,  0.2934,  0.4641]]),
 tensor([[-0.9799,  0.5412, -1.1007, -0.8789, -0.1437]]),
 tensor([-6.4585]))